In [36]:
##image processing script:
##1. call our revised YOLO (you only look once) to extract bounding boxes for objects in the iamges
##2. split the original image dataset into a set of singal-object images 
##3. extract the features of using ResNet50
##4. cluster the images using k-means
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
#import imageio
#imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline


In [32]:
#imagePath = './test_images/test1.jpg'
imagePath='data/80.jpg'
image = cv2.imread(imagePath)
#cv2.imwrite("test.png",image)

#image_crop = image[300:650,500:,:]
#resized = cv2.resize(image_crop,(448,448))

In [15]:
#function to read the bounding box text file and save it to a list
def init():
    bbx=open("box.txt",'r')
    bbxs=bbx.readlines()
    boxes=[]
    ori=[]
    for line in bbxs:
        out.write(line)
        w=line.split()
        my=[]
        for i in range(0,4):
            my.append(int(w[i]))
            #l r u d
        ori.append(my)
    bbx.close()
    return ori

In [18]:
#demo: show the bounding boxes of a given image 
imagePath='data/80.jpg'
image = cv2.imread(imagePath)
os.system("./darknet detect cfg/yolo.cfg yolo.weights "+imagePath)
my=init()
for x in my:
    cv2.rectangle(icv, (x[0], x[2]), (x[1], x[3]), (255,0,0), 3))
    cv2.imwrite("result/example.jpg",icv)
print(my)

[[507, 601, 194, 240], [308, 428, 208, 395], [360, 613, 99, 248], [1, 208, 205, 424]]


In [43]:
#predict bounding boxes and save the bounding boxes pictures to results/ folder
import os
files=os.listdir("a/")
out=open("boxes.txt","w")
for name in files:
    imagePath='a/'+name
    image = cv2.imread(imagePath)
    os.system("./darknet detect cfg/yolo.cfg yolo.weights "+imagePath)
    my=init()
    resized = cv2.resize(image,(608,608))
    icv=resized
    i=0
    for x in my:
        i=i+1
        #cv2.rectangle(icv, (x[0], x[2]), (x[1], x[3]), (255,0,0), 3)
        cv2.imwrite('results/'+name+'_'+str(i)+".jpg",icv[x[2]:x[3],x[0]:x[1]])
    out.write(name+'\t'+str(i)+'\n')
    #cv2.imwrite("result/"+name,icv)
out.close()

# get the representation of all bounding box pictures

In [48]:
from keras.objectives import mean_squared_error, categorical_crossentropy
from keras.layers import Input, Convolution2D, Dropout, GlobalAveragePooling2D
from keras.layers import Flatten, Dense, GlobalMaxPooling2D
from keras.models import Model
from keras.applications.resnet50 import ResNet50

In [57]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os.path as op
from scipy.misc import imread, imresize
from keras.applications.imagenet_utils import preprocess_input
import tarfile

In [71]:
#extract features of the bounding box pictures
from scipy import misc
from keras.preprocessing import image

model = ResNet50(weights='imagenet', include_top=False)
files=os.listdir('results')
#out=open("out_id.txt",'r')
img_rep=open("img_rep.txt",'w')
#print (len(out))
features=[]
#text=out.readlines()
m=0

for id_num in files:
    m=m+1
    img_path="results/"+id_num

    #print(img_path)
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    #img_rep.write(str(feature)+'\n')
    features.append(feature[0][0][0])
    
out.close()
img_rep.close()

1
2
3
4


In [63]:
# confirm image number
print(len(features))

4


In [64]:
#transfer it to numpyarray
f=np.array(features)

## K-means clustering

In [66]:
from sklearn.cluster import KMeans
kmeans=KMeans(n_clusters=100,random_state=0).fit(f)

In [70]:
a=kmeans.labels_
out=open("clusters.txt","w")
i=0
for id_num in files:
    label=a[i]
    sup=id_num.find(".jpg")
    pic=id_num[:sup]
    out.write(pic+"\t"+label+"\n")
    i+=1
out.close()